Thêm thư viện

In [1]:
import csv
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    ConfusionMatrixDisplay,
    confusion_matrix,
    f1_score,
    accuracy_score,
    precision_score,
    recall_score,
    classification_report
)
import pickle

import joblib
from sklearn.preprocessing import MinMaxScaler
from imblearn.under_sampling import RandomUnderSampler
import matplotlib.pyplot as plt
import seaborn as sns


Xử lí dữ liệu

In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

# Load dữ liệu
data = pd.read_csv("Car_Insurance_DaXuLy.csv")

print("Số thuộc tính: ", len(data.columns))
n = len(data.columns)
print(data)

# Lấy danh sách các cột categorical
categorical_cols = data.select_dtypes(include=['object']).columns

# Lặp qua các cột categorical và in số lượng giá trị khác nhau
for col in categorical_cols:
    print(f"Thuộc tính '{col}':")
    value_counts = data[col].value_counts()
    for value, count in value_counts.items():
        print(f"  {value}: {count}")
    print()

# In tên thuộc tính
ds_thuoctinh = list(data.columns)
print("Các thuộc tính của tập dữ liệu: ", ds_thuoctinh)

cac_nhan = list(np.unique(data.iloc[:, -1]))
print("Cột nhãn: ", list(data.columns[-1:]))
print("Các nhãn: ", cac_nhan)

# Điếm giá trị của cột nhãn
print(data.iloc[:, -1].value_counts())

# Giá trị thiếu
print("Missing value:")
print(data.isnull().sum())  # Sửa chỗ này để kiểm tra giá trị thiếu

# Tiền xử lý dữ liệu
list_dt = []
le = preprocessing.LabelEncoder()

for columnName in categorical_cols:
    le.fit(list(data[columnName]))
    nhan_cot = set(list(data[columnName]))
    data[columnName] = le.transform(list(data[columnName]))
    dic_dt = {i: int(le.transform([i])[0]) for i in nhan_cot}
    list_dt.append(dic_dt)

X_data = data.iloc[:, :-1]
y_target = data.iloc[:, -1]

for i in range(len(list_dt)):
    print(ds_thuoctinh[i])
    print(list_dt[i])

print(data)

# Lưu kết quả vào file mới
data.to_csv('Car_Insurance_numbered.csv', index=False)


Số thuộc tính:  18
        AGE     SEX      RACE DRIVING_EXPERIENCE    EDUCATION         INCOME  \
0       65+  female  majority               0-9y  high school    upper class   
1     16-25    male  majority               0-9y         none        poverty   
2     16-25  female  majority               0-9y  high school  working class   
3     16-25    male  majority               0-9y   university  working class   
4     26-39    male  majority             10-19y         none  working class   
...     ...     ...       ...                ...          ...            ...   
9980  26-39  female  majority             10-19y   university    upper class   
9981  26-39  female  majority             10-19y         none   middle class   
9982  26-39    male  majority               0-9y  high school   middle class   
9983  26-39  female  majority             10-19y  high school        poverty   
9984  26-39  female  majority               0-9y         none  working class   

      CREDIT_SCORE  

Load dữ liệu tập đã được xử lý

In [3]:
import pandas as pd
import numpy as np

# Load the dataset
def loadData(path):
    data = pd.read_csv(path)
    np.random.shuffle(data.to_numpy())  # Shuffle the data
    return data

dulieuLoad = loadData("Car_Insurance_numbered.csv")

# Tách features và target
X = dulieuLoad.iloc[:, :-1]  # Các cột đặc trưng
y = dulieuLoad.iloc[:, -1]  # Cột nhãn

# Chuyển đổi giá trị nhãn từ float thành int
y = y.astype(int)

# Chuyển đổi lại thành DataFrame để dễ thao tác
df = pd.DataFrame(X, columns=dulieuLoad.columns[:-1])
df['OUTCOME'] = y

# Số lượng nhãn 0 và 1
num_zeros = np.sum(df['OUTCOME'] == 0)
num_ones = np.sum(df['OUTCOME'] == 1)

# In số lượng dòng dữ liệu ban đầu
print("Số lượng dòng dữ liệu nhãn 0:", num_zeros)
print("Số lượng dòng dữ liệu nhãn 1:", num_ones)

# Lấy ngẫu nhiên số lượng dòng dữ liệu nhãn '0' bằng với số lượng nhãn '1'
df_zeros = df[df['OUTCOME'] == 0].sample(n=num_ones, random_state=42)

# Ghép toàn bộ dữ liệu nhãn '1' và nhãn '0' đã được undersampling
df_balanced = pd.concat([df[df['OUTCOME'] == 1], df_zeros])

# Shuffle lại dữ liệu sau khi cân bằng
df_balanced = df_balanced.sample(frac=1).reset_index(drop=True)

# Lưu dữ liệu sau khi cân bằng vào file CSV
df_balanced.to_csv('Car_Insurance_balanced.csv', index=False)

# In số dòng của tập dữ liệu sau khi cân bằng
print("Số dòng của tập dữ liệu sau khi cân bằng:", len(df_balanced))

# Kiểm tra lại số lượng nhãn '0' và '1' sau khi cân bằng
num_yes = np.sum(df_balanced['OUTCOME'] == 1)
num_no = np.sum(df_balanced['OUTCOME'] == 0)

if num_yes == num_no:
    print("Số lượng nhãn 'yes' và 'no' sau khi cân bằng là bằng nhau.")
else:
    print("Số lượng nhãn 'yes' và 'no' sau khi cân bằng không bằng nhau.")


Số lượng dòng dữ liệu nhãn 0: 6863
Số lượng dòng dữ liệu nhãn 1: 3122
Số dòng của tập dữ liệu sau khi cân bằng: 6244
Số lượng nhãn 'yes' và 'no' sau khi cân bằng là bằng nhau.


In [4]:
dulieuLoad = loadData("Car_Insurance_balanced.csv")

# Tách features và target
X = dulieuLoad.iloc[:, :-1]  # Các cột đặc trưng
y = dulieuLoad.iloc[:, -1]  # Cột nhãn

# Chuyển đổi giá trị nhãn từ float thành int
y = y.astype(int)

# Chuyển đổi lại thành DataFrame để dễ thao tác
df = pd.DataFrame(X, columns=dulieuLoad.columns[:-1])
df['OUTCOME'] = y

counts = df['OUTCOME'].value_counts()
print("Số lượng dòng dữ liệu là 0:", counts.get(0, 0))
print("Số lượng dòng dữ liệu là 1:", counts.get(1, 0))

Số lượng dòng dữ liệu là 0: 3122
Số lượng dòng dữ liệu là 1: 3122
